<a href="https://colab.research.google.com/github/little22ww/-uni-_git_hw/blob/main/Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

%%bash

# install required system dependencies
apt-get install -y xvfb x11-utils

# install required python dependencies (might need to install additional gym extras depending)
pip install gym[box2d]==0.17.* pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libxxf86dga1
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libxxf86dga1 x11-utils xvfb
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 994 kB of archives.
After this operation, 2,981 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 x11-utils amd64 7.7+3build1 [196 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.9 [784 kB]
Fetched 994 kB in 1s (1,282 kB/s)
Selecting previously unselected package libxxf86dga1:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../libxxf86dga1_2%3a1.1.4-1_amd64.deb ...
Unpacking libxxf86dga1:amd64 (2:1.1.4-1) ...
S

In [ ]:
pip install nevergrad

     |████████████████████████████████| 411 kB 5.1 MB/s 
     |████████████████████████████████| 269 kB 52.7 MB/s 
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=982ebfc35bd70251fea5a070c4a83e278da09c371f2120976089e4bc0778f2fb
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [ ]:
import pyvirtualdisplay


_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    size=(1400, 900))
_ = _display.start()

In [ ]:
import gym
import numpy as np
#import nevergrad as ng
import cv2
from scipy.optimize import minimize
import scipy
import scipy.stats as st
from scipy.linalg import *

import matplotlib.pyplot as plt 
from matplotlib import animation 
import imageio

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

In [ ]:
#N = 200#sample size for gaussian smoothing
#n = 200#dimension of the problem

#256 is for haramard
n = 256
N = 256

sigma = 0.1
T = 200

In [ ]:
#noise,n:dimension, N:sample size for gaussian smoothing
def independent_gau(n,N,sigma = 0.1):
    res = np.empty((N,n))
    for i in range(N):
        res[i,:] = np.random.multivariate_normal(mean = np.zeros(n), cov = sigma*np.eye(n)) 
    return res

def orth_gau(n,N,sigma = 0.1):
    res = independent_gau(n,N,sigma)
    ort = orth(res.T)
    leng = np.linalg.norm(ort, axis=1, keepdims=True)
    return ort/leng

#hadamard 
def hadamard(n,T):
    H1 = np.array([[1/np.sqrt(2),1/np.sqrt(2)],[1/np.sqrt(2),-1/np.sqrt(2)]])
    H = H1
    l = 2
    while 2*l<=n:
        H = np.kron(H,H1)
        l*=2
    n_h = H.shape[0]
    
    M = np.eye(n_h)
    for i in range(T):
        vec=np.random.choice([-1,1],n_h)
        D=np.diag(vec)
        temp = np.dot(H,D)
        M = np.dot(M,temp)
    return M


def givens(n,i,j,theta):
    G = np.eye(n)
    G[i][i] = G[j][j] = np.cos(theta)
    G[i][j] = -np.sin(theta)
    G[j][i] = np.sin(theta)
    return G

def random_givens(n,T):
    K = np.eye(n)
    for t in range(T):
        i = np.random.randint(0,n)
        j = np.random.randint(0,n)
        theta = 2*np.pi*np.random.rand()
        K = np.dot(K,givens(n,i,j,theta))
    return K

In [ ]:
#original function
def fun(a_list):
    env = gym.make('CartPole-v0')
    total_reward = 0.0
    total_steps = 0
    obs = env.reset()
   
    for a in a_list:
        a = 0 if a<=0.5 else 1
        obs, reward, done, _ = env.step(a) # take a random action
        total_reward += reward
        total_steps += 1
        if done:
            break
    #print("Episode done in %d steps, total reward %.2f" % (total_steps, total_reward))
    return -total_reward

#gaussian smoothing
def gau_smo(a_list):
    reward_list = np.zeros(N)
    n = len(a_list)
    
    #generate noise with different methods
    noise = independent_gau(n,N)
    #noise = orth_gau(n,N)
    
    for i in range(N):
        curr_noise = noise[i,:]
        reward_list[i] = fun(a_list+curr_noise)
    return reward_list.mean()

In [ ]:
# monte carlo based gradient
def vanilla(a_list):
    n = len(a_list)
    grad = np.zeros(n)
    
    #generate noise with different methods
    #noise = independent_gau(n,N)
    #noise = orth_gau(n,N)
    noise = hadamard(n,T)
    #noise = random_givens(n,T)
    
    for i in range(noise.shape[0]):
        curr_noise = noise[i,:]       
        theta = a_list+sigma*curr_noise
        F = fun(theta)
        grad+=F*curr_noise
    grad = grad/(N*sigma)
    return grad

def antithetic(a_list):
    n = len(a_list)
    grad = np.zeros(n)
    
    #generate noise with different methods
    #noise = independent_gau(n,N)
    #noise = orth_gau(n,N)
    noise = hadamard(n,T)
    #noise = random_givens(n,T)
    
    for i in range(noise.shape[0]):
        curr_noise = noise[i,:]             
        theta1 = a_list+sigma*curr_noise
        theta2 = a_list-sigma*curr_noise
        F1,F2 = fun(theta1),fun(theta2)
        grad+=(F1-F2)*curr_noise
    grad = grad/(2*N*sigma)
    return grad

def FD(a_list):
    n = len(a_list)
    grad = np.zeros(n)
    F = fun(a_list)
    
    #generate noise with different methods
    #noise = independent_gau(n,N)
    #noise = orth_gau(n,N)
    noise = hadamard(n,T)
    #noise = random_givens(n,T)
    
    for i in range(noise.shape[0]):
        curr_noise = noise[i,:]              
        theta = a_list+sigma*curr_noise
        F1 = fun(theta)
        grad+=(F1-F)*curr_noise
    grad = grad/(N*sigma)
    return grad

In [ ]:
#regression based gradient
def lasso_gra(a_list):
    F = fun(a_list)
    y = np.zeros(N)
    #generate noise with different methods
    noise = independent_gau(n,N)
    #noise = orth_gau(n,N)
    for i in range(N):
        y[i] = fun(a_list+noise[i,:])-F
    
    SSmodel=StandardScaler()
    SSmodel.fit(noise)
    Train_x=SSmodel.transform(noise)
    model = Lasso(fit_intercept = False)
    model.fit(Train_x,y)
    return model.coef_/np.std(noise,axis = 0)

def ridge_gra(a_list):
    F = fun(a_list)
    y = np.zeros(N)
    #generate noise with different methods
    noise = independent_gau(n,N)
    #noise = orth_gau(n,N)
    for i in range(N):
        y[i] = fun(a_list+noise[i,:])-F
    
    SSmodel=StandardScaler()
    SSmodel.fit(noise)
    Train_x=SSmodel.transform(noise)
    model = Ridge(fit_intercept = False)
    model.fit(Train_x,y)
    return model.coef_/np.std(noise,axis = 0)

def lp_gra(a_list):
    F = fun(a_list)
    y = np.zeros(N)
    #generate noise with different methods
    noise = independent_gau(n,N)
    #noise = orth_gau(n,N)
    for i in range(N):
        y[i] = fun(a_list+noise[i,:])-F
    
    def lp_decoding(z):
        res=np.abs(y - np.dot(noise[i,:],z))
        return np.sum(res)
    
    curr_opt = scipy.optimize.minimize(lp_decoding,np.zeros(n),tol = 0.01)
    return curr_opt.x

In [ ]:
R = np.random.RandomState(seed = 10)
initial = R.rand(n)
initial_reward = fun(initial)
print('initial reward',initial_reward)

initial reward -28.0


In [ ]:
#optimization part, conjugate gradient method
bound = [[0,1]]*n
df = pd.DataFrame(columns = ['fun','nit','nfev','njev'])
for i in range(200):
  opt = scipy.optimize.minimize(fun,initial,bounds =bound,method = 'CG',jac = FD,tol = 0.01,options = {'disp' : True} )
  df = df.append({'fun':-opt.fun,'nit':opt.nit,'nfev':opt.nfev,'njev':opt.njev},ignore_index = True)

/usr/local/lib/python3.7/dist-packages/scipy/optimize/_minimize.py:522: RuntimeWarning: Method CG cannot handle constraints nor bounds.
  RuntimeWarning)


         Current function value: -64.000000
         Iterations: 3
         Function evaluations: 34
         Gradient evaluations: 21


In [ ]:
#turn the optimization output into the action space
a_vanilla_indepent = opt.x
a_vanilla_indepent[a_vanilla_indepent>0.5]=1
a_vanilla_indepent[a_vanilla_indepent<=0.5]=0

In [ ]:
#generate gif
def saveanimation(frames,address="./movie.gif"):
  """ 
  This method ,given the frames of images make the gif and save it in the folder
  
  params:
      frames:method takes in the array or np.array of images
      address:(optional)given the address/location saves the gif on that location
              otherwise save it to default address './movie.gif'
  
  return :
      none
  """
  imageio.mimsave(address, frames)

In [ ]:
#see the optimal result,generate gif
env = gym.make('CartPole-v0')
env.reset()
frames = []

for a in a_vanilla_indepent:
    a = int(a)
    frames.append(env.render(mode = 'rgb_array'))
    obs,reward,done,info = env.step(a) # take a random action
    if done:
        break
env.close()
## save the images as array
saveanimation(frames,address="./vani_ind.gif")
